In [1]:
from backend.engine.core.model import EconomyModel
model = EconomyModel(num_people=3)
for _ in range(5):  # run 5 weeks
    model.step()
    print(f"Week {model.week} completed")

Industry Type: groceries, Starting Price: 10.0
Industry Type: utilities, Starting Price: 10.0
Industry Type: automobiles, Starting Price: 10.0
Industry Type: housing, Starting Price: 10.0
Industry Type: household_goods, Starting Price: 10.0
Industry Type: entertainment, Starting Price: 10.0
Industry Type: luxury, Starting Price: 10.0
Income: 101
Income: 101
Income: 101
Week 1 completed
Industry Type: groceries, Starting Price: 10.0
Industry Type: utilities, Starting Price: 10.0
Industry Type: automobiles, Starting Price: 10.0
Industry Type: housing, Starting Price: 10.0
Industry Type: household_goods, Starting Price: 10.0
Industry Type: entertainment, Starting Price: 10.0
Industry Type: luxury, Starting Price: 10.0
Income: 102
Income: 102
Income: 102
Week 2 completed
Industry Type: groceries, Starting Price: 10.0
Industry Type: utilities, Starting Price: 10.0
Industry Type: automobiles, Starting Price: 10.0
Industry Type: housing, Starting Price: 10.0
Industry Type: household_goods, St